In [9]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from pypalettes import load_cmap
from matplotlib.font_manager import FontProperties
from matplotlib.patches import Patch
from matplotlib.patches import FancyArrowPatch
from highlight_text import fig_text, ax_text

df = pd.read_csv('GCB2022v27_percapita_flat.csv')
df = df[df['Year']==df['Year'].max()]
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
world = world[world['continent'] == 'Europe']
data = world.merge(df, how='left', left_on='name', right_on='Country')
data = data[~data['Country'].isin(['Russia', 'Iceland'])]
data = data[['Country', 'Total', 'geometry']]
data = data.dropna()
data = data.sort_values('Total', ascending=False)
data.columns = ['country', 'co2Capita', 'geometry']
data.to_file('data.geojson', driver='GeoJSON')
data.sample(3)

/var/folders/m2/jfzxyg4s0xz3jyw5f94r7tcm0000gn/T/ipykernel_70412/3272479112.py:12: FutureWarning: The geopandas.dataset module is deprecated and will be removed in GeoPandas 1.0. You can get the original 'naturalearth_lowres' data from https://www.naturalearthdata.com/downloads/110m-cultural-vectors/.
  world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))


,country,co2Capita,geometry
25,Ireland,7.528353,"POLYGON ((-6.19788 53.86757, -6.03299 53.15316..."
26,Italy,5.548371,"MULTIPOLYGON (((10.44270 46.89355, 11.04856 46..."
32,Slovakia,6.481400,"POLYGON ((22.55814 49.08574, 22.28084 48.82539..."


In [14]:
def draw_arrow(tail_position, head_position, invert=False, radius=0.5, color='black', fig=None):
    if fig is None:
        fig = plt.gcf()
    kw = dict(arrowstyle="Simple, tail_width=0.5, head_width=4, head_length=8", color=color, lw=0.5)
    if invert:
        connectionstyle = f"arc3,rad=-{radius}"
    else:
        connectionstyle = f"arc3,rad={radius}"
    a = FancyArrowPatch(
        tail_position, head_position,
        connectionstyle=connectionstyle,
        transform=fig.transFigure,
        **kw
    )
    fig.patches.append(a)

def create_legend_elements(cmap, value_ranges, labels):
   legend_elements = []
   n = len(value_ranges)
   for i in range(n-1):
      value = (value_ranges[i] + value_ranges[i+1]) / 2 / value_ranges[-1]  # Normalize the value to [0, 1]
      legend_elements.append(Patch(facecolor=cmap(value), label=labels[i]))
   return legend_elements

In [ ]:
# load the colormap
cmaps = ['BrwnYl', 'YlGnBu', 'Concha']
cmap = load_cmap(cmaps[0], type='continuous')
background_color = 'white'
text_color = 'black'

value_ranges = [1, 3, 5, 7, 9, 11, 13]
labels = ['0-2 t', '2-4 t', '4-6 t', '6-8 t', '8-10 t', '10-12 t', '12+ t']

legend_elements = create_legend_elements(cmap, value_ranges, labels)

# load the font
personal_path = '/Users/josephbarbier/Library/Fonts/'
font = FontProperties(fname=personal_path + 'BebasNeue-Regular.ttf')
other_font = FontProperties(fname=personal_path + 'FiraSans-Light.ttf')
other_bold_font = FontProperties(fname=personal_path + 'FiraSans-Medium.ttf')

# initialize the figure
fig, ax = plt.subplots(figsize=(10, 10), dpi=300)
fig.set_facecolor(background_color)

# create the plot
data.plot(ax=ax, column='co2Capita', cmap=cmap, edgecolor='black', linewidth=0.5)

# custom axis
ax.set_xlim(-11, 41)
ax.set_ylim(32, 73)
ax.set_axis_off()

# add the patches to the plot area as a legend
rectangle_width = 2
rectangle_height = 1.5
legend_x = 35
legend_y_start = 65
legend_y_step = 1.5
for i, element in enumerate(legend_elements):
   ax.add_patch(plt.Rectangle((legend_x, legend_y_start - i * legend_y_step), rectangle_width, rectangle_height,
                              color=element.get_facecolor(), ec='black', lw=0.6))
   ax.text(legend_x + 2.5, legend_y_start - i * legend_y_step + 0.7, element.get_label(), 
            fontsize=12, fontproperties=other_font, color=text_color, va='center')

# compute centroids and display the total
adjustments = {
    'France': (10, 3),
    'Italy': (-2.4, 2.5),
    'Lithuania': (0, -0.6),
    'Finland': (0, -2),
    'Romania': (0, 0),
    'Belarus': (0, -0.4),
    'Bulgaria': (0, 0),
    'Greece': (-1.2, -0.8),
    'Croatia': (0, -1),
    'Cyprus': (0, -1),
    'Ireland': (0, -1),
    'Malta': (0, -1),
    'Slovenia': (0, -1),
    'Slovakia': (-0.7, -0.8),
    'Estonia': (0, -0.7),
    'Latvia': (0, -0.5),
    'Belgium': (0, -0.7),
    'Austria': (0, -0.4),
    'Spain': (0, 0),
    'Portugal': (-0.5, -1),
    'Luxembourg': (0, -1),
    'Germany': (-0.2, 0),
    'Hungary': (-0.3, -1),
    'Czechia': (0, -1),
    'Poland': (0, 0.2),
    'Sweden': (-1.2, -2.8),
    'Denmark': (0, -1),
    'Netherlands': (0, 0),
    'United Kingdom': (1, -1.5),
    'Switzerland': (0, -0.5),
    'Norway': (-4, -5.5),
}
data_projected = data.to_crs(epsg=3035)
data_projected['centroid'] = data_projected.geometry.centroid
data['centroid'] = data_projected['centroid'].to_crs(data.crs)
countries_to_annotate = data['country'].tolist()
countries_to_annotate = [
   'France', 'Italy', 'Romania', 'Poland',
   'Finland', 'Ukraine',
   'Spain', 'Germany', 
   'Sweden', 'United Kingdom',
   'Belarus', 'Norway'
]
for country in countries_to_annotate:
   centroid = data.loc[data['country'] == country, 'centroid'].values[0]
   x, y = centroid.coords[0]
   try:
      x += adjustments[country][0]
      y += adjustments[country][1]
   except KeyError:
      pass
   rate = data.loc[data['country'] == country, 'co2Capita'].values[0]
   if country=='United Kingdom':
      country = 'UK'
   if rate > 7:
      color_text = 'white'
   else:
      color_text = text_color
   ax_text(
      x=x, y=y, s=f"<{country.upper()}>: {rate:.2f}", fontsize=9, font=other_font, color=color_text,
      ha='center', va='center', ax=ax, highlight_textprops=[{'font': other_bold_font}]
   )

# title
fig_text(
   s="CO2 emissions per capita in Europe (2021)", x=0.5, y=0.14,
   color=text_color, fontsize=25, font=font, ha='center', va='top', ax=ax
)

# subtitle
fig_text(
   s="<Unit>: metric tons | <Data>: zenodo.org | <Viz>: barbierjoseph.com", x=0.5, y=0.1,
   color=text_color, fontsize=14, font=other_font, ha='center', va='top', ax=ax,
   highlight_textprops=[{'font': other_bold_font}, {'font': other_bold_font}, {'font': other_bold_font}]
)

# arrows for the Luxembourg
luxembourg_values = data.loc[data['country'] == 'Luxembourg', 'co2Capita'].values[0]
draw_arrow((0.32, 0.7), (0.375, 0.45), radius=0.3, fig=fig)
fig_text(
   s=f"<LUXEMBOURG>: {luxembourg_values:.2f} t", x=0.32, y=0.71, highlight_textprops=[{'font': other_bold_font}],
   color=text_color, fontsize=9, font=other_font, ha='center', va='center', fig=fig
)

# display the plot
plt.savefig('map.png', dpi=300)
plt.show()